In [1]:
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium import webdriver
import time
import pandas as pd

In [2]:
def get_jobs(keyword, num_jobs, verbose,path,slp_time):
    
    '''Gathers jobs as a dataframe, scraped from Glassdoor'''
    
    #Initializing the webdriver
    options = webdriver.ChromeOptions()
    
    #Uncomment the line below if you'd like to scrape without a new Chrome window every time.
    #options.add_argument('headless')
    
    #Change the path to where chromedriver is in your home folder.
    driver = webdriver.Chrome(executable_path=path, options=options)
    driver.set_window_size(1120, 1000)
    url = "https://www.glassdoor.com/Job/jobs.htm?suggestCount=0&suggestChosen=false&clickSource=searchBtn&typedKeyword="+keyword+"&sc.keyword="+keyword+"&locT=&locId=&jobType="
    #url = 'https://www.glassdoor.com/Job/jobs.htm?sc.keyword="' + keyword + '"&locT=C&locId=1147401&locKeyword=San%20Francisco,%20CA&jobType=all&fromAge=-1&minSalary=0&includeNoSalaryJobs=true&radius=100&cityId=-1&minRating=0.0&industryId=-1&sgocId=-1&seniorityType=all&companyId=-1&employerSizes=0&applicationType=0&remoteWorkType=0'
    driver.get(url)
    jobs = []

    while len(jobs) < num_jobs:  #If true, should be still looking for new jobs.

        #Let the page load. Change this number based on your internet speed.
        #Or, wait until the webpage is loaded, instead of hardcoding it.
        time.sleep(slp_time)

        #Test for the "Sign Up" prompt and get rid of it.
        try:
            driver.find_element_by_class_name("selected").click()
        except ElementClickInterceptedException:
            pass

        time.sleep(.1)

        try:
            driver.find_element_by_css_selector('[alt="Close"]').click()  #clicking to the X.
        except NoSuchElementException:
            pass

        
        #Going through each job in this page
        job_buttons = driver.find_elements_by_class_name("jl")  #jl for Job Listing. These are the buttons we're going to click.
        for job_button in job_buttons:  

            print("Progress: {}".format("" + str(len(jobs)) + "/" + str(num_jobs)))
            if len(jobs) >= num_jobs:
                break

            job_button.click()  #You might 
            time.sleep(1)
            collected_successfully = False
            
            while not collected_successfully:
                try:
                    company_name = driver.find_element_by_xpath('.//div[@class="employerName"]').text
                    location = driver.find_element_by_xpath('.//div[@class="location"]').text
                    job_title = driver.find_element_by_xpath('.//div[contains(@class, "title")]').text
                    job_description = driver.find_element_by_xpath('.//div[@class="jobDescriptionContent desc"]').text
                    collected_successfully = True
                except:
                    time.sleep(5)

            try:
                salary_estimate = driver.find_element_by_xpath('.//span[@class="gray salary"]').text
            except NoSuchElementException:
                salary_estimate = -1 #You need to set a "not found value. It's important."
            
            try:
                rating = driver.find_element_by_xpath('.//span[@class="rating"]').text
            except NoSuchElementException:
                rating = -1 #You need to set a "not found value. It's important."

            #Printing for debugging
            if verbose:
                print("Job Title: {}".format(job_title))
                print("Salary Estimate: {}".format(salary_estimate))
                print("Job Description: {}".format(job_description[:500]))
                print("Rating: {}".format(rating))
                print("Company Name: {}".format(company_name))
                print("Location: {}".format(location))

            #Going to the Company tab...
            #clicking on this:
            #<div class="tab" data-tab-type="overview"><span>Company</span></div>
            try:
                driver.find_element_by_xpath('.//div[@class="tab" and @data-tab-type="overview"]').click()

                try:
                    #<div class="infoEntity">
                    #    <label>Headquarters</label>
                    #    <span class="value">San Francisco, CA</span>
                    #</div>
                    headquarters = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Headquarters"]//following-sibling::*').text
                except NoSuchElementException:
                    headquarters = -1

                try:
                    size = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Size"]//following-sibling::*').text
                except NoSuchElementException:
                    size = -1

                try:
                    founded = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Founded"]//following-sibling::*').text
                except NoSuchElementException:
                    founded = -1

                try:
                    type_of_ownership = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Type"]//following-sibling::*').text
                except NoSuchElementException:
                    type_of_ownership = -1

                try:
                    industry = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Industry"]//following-sibling::*').text
                except NoSuchElementException:
                    industry = -1

                try:
                    sector = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Sector"]//following-sibling::*').text
                except NoSuchElementException:
                    sector = -1

                try:
                    revenue = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Revenue"]//following-sibling::*').text
                except NoSuchElementException:
                    revenue = -1

                try:
                    competitors = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Competitors"]//following-sibling::*').text
                except NoSuchElementException:
                    competitors = -1

            except NoSuchElementException:  #Rarely, some job postings do not have the "Company" tab.
                headquarters = -1
                size = -1
                founded = -1
                type_of_ownership = -1
                industry = -1
                sector = -1
                revenue = -1
                competitors = -1

                
            if verbose:
                print("Headquarters: {}".format(headquarters))
                print("Size: {}".format(size))
                print("Founded: {}".format(founded))
                print("Type of Ownership: {}".format(type_of_ownership))
                print("Industry: {}".format(industry))
                print("Sector: {}".format(sector))
                print("Revenue: {}".format(revenue))
                print("Competitors: {}".format(competitors))
                print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")

            jobs.append({"Job Title" : job_title,
            "Salary Estimate" : salary_estimate,
            "Job Description" : job_description,
            "Rating" : rating,
            "Company Name" : company_name,
            "Location" : location,
            "Headquarters" : headquarters,
            "Size" : size,
            "Founded" : founded,
            "Type of ownership" : type_of_ownership,
            "Industry" : industry,
            "Sector" : sector,
            "Revenue" : revenue,
            "Competitors" : competitors})
            #add job to jobs

        #Clicking on the "next page" button
        try:
            driver.find_element_by_xpath('.//li[@class="next"]//a').click()
        except NoSuchElementException:
            print("Scraping terminated before reaching target number of jobs. Needed {}, got {}.".format(num_jobs, len(jobs)))
            break

    return pd.DataFrame(jobs)  #This line converts the dictionary object into a pandas DataFrame.

In [3]:
#This line will open a new chrome window and start the scraping.
path = "C:/Users/ERP80/Documents/ds_salary_proj/chromedriver"
df = get_jobs('data scientist',100,False,path,15)


Progress: 0/100
Progress: 1/100
Progress: 2/100
Progress: 3/100
Progress: 4/100
Progress: 5/100
Progress: 6/100
Progress: 7/100
Progress: 8/100
Progress: 9/100
Progress: 10/100
Progress: 11/100
Progress: 12/100
Progress: 13/100
Progress: 14/100
Progress: 15/100
Progress: 16/100
Progress: 17/100
Progress: 18/100
Progress: 19/100
Progress: 20/100
Progress: 21/100
Progress: 22/100
Progress: 23/100
Progress: 24/100
Progress: 25/100
Progress: 26/100
Progress: 27/100
Progress: 28/100
Progress: 29/100
Progress: 30/100
Progress: 31/100
Progress: 32/100
Progress: 33/100
Progress: 34/100
Progress: 35/100
Progress: 36/100
Progress: 37/100
Progress: 38/100
Progress: 39/100
Progress: 40/100
Progress: 41/100
Progress: 42/100
Progress: 43/100
Progress: 44/100
Progress: 45/100
Progress: 46/100
Progress: 47/100
Progress: 48/100
Progress: 49/100
Progress: 50/100
Progress: 51/100
Progress: 52/100
Progress: 53/100
Progress: 54/100
Progress: 55/100
Progress: 56/100
Progress: 57/100
Progress: 58/100
Progres

In [4]:
df.head()

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors
0,Staff Data Scientist,CA$73K-CA$100K (Glassdoor Est.),"At realtor.com, we process terabytes of data e...",2.7,realtor.com\n2.7,Vancouver,"Santa Clara, CA",1001 to 5000 employees,1993,Company - Private,Internet,Information Technology,Unknown / Non-Applicable,"Zillow, Trulia, Redfin"
1,Senior Data Analyst,CA$73K-CA$100K (Glassdoor Est.),Job Overview:\n\nThe Senior Data Analyst is a ...,3.4,Teranet\n3.4,Toronto,"Toronto, Canada",501 to 1000 employees,1991,Company - Private,Computer Hardware & Software,Information Technology,$100 to $500 million (CAD),-1
2,Game Data Scientist,CA$73K-CA$100K (Glassdoor Est.),"Prodigy Education connects students, parents, ...",4.4,Prodigy Game\n4.4,Oakville,"Oakville, Canada",201 to 500 employees,2011,Company - Private,Computer Hardware & Software,Information Technology,Unknown / Non-Applicable,-1
3,Data Engineer/Backend Developer,CA$73K-CA$100K (Glassdoor Est.),Data Engineer/Backend Developer - Downtown Van...,-1,Label Engine,Vancouver,"Los Angeles, CA",51 to 200 employees,-1,Company - Private,-1,-1,Unknown / Non-Applicable,-1
4,Data Scientist,CA$73K-CA$100K (Glassdoor Est.),"Company Profile\n\nHome Capital Group Inc., to...",3.1,Home Trust\n3.1,Toronto,"Toronto, Canada",501 to 1000 employees,1977,Company - Public,Lending,Finance,$100 to $500 million (CAD),-1


In [5]:
df

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors
0,Staff Data Scientist,CA$73K-CA$100K (Glassdoor Est.),"At realtor.com, we process terabytes of data e...",2.7,realtor.com\n2.7,Vancouver,"Santa Clara, CA",1001 to 5000 employees,1993,Company - Private,Internet,Information Technology,Unknown / Non-Applicable,"Zillow, Trulia, Redfin"
1,Senior Data Analyst,CA$73K-CA$100K (Glassdoor Est.),Job Overview:\n\nThe Senior Data Analyst is a ...,3.4,Teranet\n3.4,Toronto,"Toronto, Canada",501 to 1000 employees,1991,Company - Private,Computer Hardware & Software,Information Technology,$100 to $500 million (CAD),-1
2,Game Data Scientist,CA$73K-CA$100K (Glassdoor Est.),"Prodigy Education connects students, parents, ...",4.4,Prodigy Game\n4.4,Oakville,"Oakville, Canada",201 to 500 employees,2011,Company - Private,Computer Hardware & Software,Information Technology,Unknown / Non-Applicable,-1
3,Data Engineer/Backend Developer,CA$73K-CA$100K (Glassdoor Est.),Data Engineer/Backend Developer - Downtown Van...,-1,Label Engine,Vancouver,"Los Angeles, CA",51 to 200 employees,-1,Company - Private,-1,-1,Unknown / Non-Applicable,-1
4,Data Scientist,CA$73K-CA$100K (Glassdoor Est.),"Company Profile\n\nHome Capital Group Inc., to...",3.1,Home Trust\n3.1,Toronto,"Toronto, Canada",501 to 1000 employees,1977,Company - Public,Lending,Finance,$100 to $500 million (CAD),-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Scientist,CA$116K-CA$162K (Glassdoor Est.),SCIENTIST- Research Institute\nThe Hospital fo...,4.1,SickKids\n4.1,Toronto,"Toronto, Canada",5001 to 10000 employees,1875,Company - Public,Healthcare Services & Hospitals,Healthcare,$1 to $2 billion (CAD),-1
96,Senior Research Associate/Data Collection Cons...,CA$116K-CA$162K (Glassdoor Est.),Research Associate III/Senior Research Associa...,3.8,Evidera\n3.8,Montreal,"Bethesda, MD",501 to 1000 employees,2013,Subsidiary or Business Segment,Biotech & Pharmaceuticals,Biotech & Pharmaceuticals,$100 to $500 million (CAD),"IQVIA, ICON"
97,Data Analyst,CA$116K-CA$162K (Glassdoor Est.),Who Are We?\n\nAt National Credit Recovery Inc...,3.2,NCRI\n3.2,Mississauga,"Mississauga, Canada",1 to 50 employees,-1,Company - Private,Investment Banking & Asset Management,Finance,$1 to $5 million (CAD),-1
98,Data Analyst,CA$116K-CA$162K (Glassdoor Est.),We are Systematix and we are currently looking...,3.8,Systematix\n3.8,Edmonton,"Montréal, Canada",501 to 1000 employees,1975,Company - Private,Consulting,Business Services,$25 to $50 million (CAD),-1


In [6]:
df.to_csv(r"C:\Users\ERP80\Documents\ds_salary_proj\glassdoor_jobs.csv", index = None, header=True)

In [7]:
df = pd.read_csv(r"C:\Users\ERP80\Documents\ds_salary_proj\glassdoor_jobs.csv", index_col = None, header=0)
print(df)

                                            Job Title  \
0                                Staff Data Scientist   
1                                 Senior Data Analyst   
2                                 Game Data Scientist   
3                     Data Engineer/Backend Developer   
4                                      Data Scientist   
..                                                ...   
95                                          Scientist   
96  Senior Research Associate/Data Collection Cons...   
97                                       Data Analyst   
98                                       Data Analyst   
99                                  Data Scientist II   

                     Salary Estimate  \
0    CA$73K-CA$100K (Glassdoor Est.)   
1    CA$73K-CA$100K (Glassdoor Est.)   
2    CA$73K-CA$100K (Glassdoor Est.)   
3    CA$73K-CA$100K (Glassdoor Est.)   
4    CA$73K-CA$100K (Glassdoor Est.)   
..                               ...   
95  CA$116K-CA$162K (Glassdoor Est.